# MODEL TRAINING

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# logistic regression 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix, classification_report)
from sklearn.model_selection import GridSearchCV


# RF
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

# Gradient Boosting Classifier 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV

# adaboost
from sklearn.ensemble import AdaBoostClassifier

In [2]:
X_train = pd.read_csv("X_train_clean.csv")
X_test = pd.read_csv("X_test_clean.csv")
y_train = pd.read_csv("y_train.csv")
y_test = pd.read_csv("y_test.csv")


In [7]:
print(f"Shape of X_train -: {X_train.shape}")
print(f"Shape of X_test -: {X_test.shape}")
print(f"Shape of y_train -: {y_train.shape}")
print(f"Shape of y_test -: {y_test.shape}")

Shape of X_train -: (40000, 69)
Shape of X_test -: (10000, 69)
Shape of y_train -: (40000,)
Shape of y_test -: (10000,)


In [4]:
y_train = y_train["Churned"]
y_test = y_test["Churned"]

In [5]:
y_test.value_counts(1)*100

Churned
0    71.1
1    28.9
Name: proportion, dtype: float64

## logistic regression model

In [9]:
# Fill NaNs with 0 (safe for scaled features)
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

# Ensure all numeric
X_train = X_train.astype(np.float64)
X_test = X_test.astype(np.float64)

In [10]:
# Initialize model = this is baseline model without handling imbalance data
log_reg = LogisticRegression(max_iter=1000, random_state=42)

# Train model
log_reg.fit(X_train, y_train)



LogisticRegression(max_iter=1000, random_state=42)

In [11]:
# Predictions
y_train_pred = log_reg.predict(X_train)
y_test_pred = log_reg.predict(X_test)

y_test_prob = log_reg.predict_proba(X_test)[:, 1] # return probability ,[:,1] return probability of positive class churn

In [12]:
# Evaluation
print("Accuracy:", accuracy_score(y_test, y_test_pred))
print("Precision:", precision_score(y_test, y_test_pred))
print("Recall:", recall_score(y_test, y_test_pred))
print("F1 Score:", f1_score(y_test, y_test_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_test_prob))

print("Confusion Matrix:\n", confusion_matrix(y_test, y_test_pred))
print( "\nClassification Report:\n",classification_report(y_test, y_test_pred))

Accuracy: 0.7659
Precision: 0.6598718695398952
Recall: 0.3920415224913495
F1 Score: 0.49186021271977426
ROC-AUC: 0.7755721995921724
Confusion Matrix:
 [[6526  584]
 [1757 1133]]

Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.92      0.85      7110
           1       0.66      0.39      0.49      2890

    accuracy                           0.77     10000
   macro avg       0.72      0.65      0.67     10000
weighted avg       0.75      0.77      0.75     10000



#### insight
- Out of all customers who actually stayed, the model correctly identified 6,526 customers (True Negatives), but wrongly marked 584 staying customers as leaving.
- Out of all customers who actually left, the model correctly caught only 1,757 customers (True Positives) and missed 1133 customers who left without warning (False Negatives).
- When the model predicts that a customer will leave, it is correct about 67% of the time (this is Precision), so its warnings are usually reliable.
- However, the model is able to catch only 43% of the customers who actually leave (this is Recall), meaning more than half of churners are missed.
- Overall, the model prefers saying “customer will stay”, which increases accuracy but reduces its ability to detect customers who are about to leave

In [13]:
lr = LogisticRegression(
    class_weight='balanced',
    max_iter=1000,
    random_state=42
)

lr.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)

In [14]:
y_prob = lr.predict_proba(X_test)[:, 1]

In [15]:
threshold = 0.4 # default threshold is 0.5 but it  is not optimal 
y_pred = (y_prob >= threshold).astype(int)
# predict_proba() → gives churn probability (risk score)
#Probabilities are used for ROC-AUC and threshold tuning

In [16]:
print("Baseline Logistic Regression (Balanced + Threshold 0.4)")
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Baseline Logistic Regression (Balanced + Threshold 0.4)
[[3906 3204]
 [ 486 2404]]
              precision    recall  f1-score   support

           0       0.89      0.55      0.68      7110
           1       0.43      0.83      0.57      2890

    accuracy                           0.63     10000
   macro avg       0.66      0.69      0.62     10000
weighted avg       0.76      0.63      0.65     10000



#### Insight
Baseline Logistic Regression (Balanced + Threshold = 0.4)

The model prioritizes identifying churned customers by using class balancing and a lower threshold. This results in high recall for churners, meaning most at-risk customers are successfully detected. Although precision and overall accuracy decrease due to more false positives, this trade-off is acceptable because missing a churner is more costly than contacting a loyal customer.

In [17]:
param_grid = {
    'C': [0.01, 0.1, 1, 10], # C controls how strong the regularization is.
    'penalty': ['l2'], # Penalty defines how the regularization is applied to model weights.(l1,l2)
    'solver': ['liblinear'] # solver is the optimization algorithm that finds the best model parameters (weights) by minimizing the loss function.
} #liblinear is chosen because it is stable, supports imbalance handling, and works well for binary classification.

grid = GridSearchCV(
    LogisticRegression(
        class_weight='balanced',
        max_iter=1000,
        random_state=42
    ),
    param_grid=param_grid,
    scoring='recall',   # focus on catching churners
    cv=5,
    n_jobs=-1
)

grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=LogisticRegression(class_weight='balanced',
                                          max_iter=1000, random_state=42),
             n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'],
                         'solver': ['liblinear']},
             scoring='recall')

In [18]:
best_lr = grid.best_estimator_ #best_estimator_ is the final trained model that performed best during GridSearch.

print("Best Parameters:", grid.best_params_)

Best Parameters: {'C': 0.01, 'penalty': 'l2', 'solver': 'liblinear'}


In [19]:
y_prob_best = best_lr.predict_proba(X_test)[:, 1]

In [20]:
threshold = 0.4
y_pred_best = (y_prob_best >= threshold).astype(int)

In [21]:
# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred_best))
print("Precision:", precision_score(y_test, y_pred_best))
print("Recall:", recall_score(y_test, y_pred_best))
print("F1 Score:", f1_score(y_test, y_pred_best))
print("ROC-AUC:", roc_auc_score(y_test, y_prob_best))

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_best))
print( "\nClassification Report:\n",classification_report(y_test, y_pred_best))

Accuracy: 0.6293
Precision: 0.4276863161621526
Recall: 0.8359861591695502
F1 Score: 0.5658742241480267
ROC-AUC: 0.7762306123740138
Confusion Matrix:
 [[3877 3233]
 [ 474 2416]]

Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.55      0.68      7110
           1       0.43      0.84      0.57      2890

    accuracy                           0.63     10000
   macro avg       0.66      0.69      0.62     10000
weighted avg       0.76      0.63      0.64     10000



#### Insight
Logistic Regression after GridSearchCV

After hyperparameter tuning with GridSearchCV, the model further improves its ability to identify churned customers, achieving very high recall for the churn class. While precision and overall accuracy decrease due to more aggressive churn predictions, the model successfully minimizes missed churners.

## Random Forest

In [22]:
rf = RandomForestClassifier(random_state=42)


In [23]:
rf.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [24]:
y_pred_rf = rf.predict(X_test)


In [25]:
# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Precision:", precision_score(y_test, y_pred_rf))
print("Recall:", recall_score(y_test, y_pred_rf))
print("F1 Score:", f1_score(y_test, y_pred_rf))

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_rf))
print("\nClassification Report:\n", classification_report(y_test, y_pred_rf))

Accuracy: 0.9139
Precision: 0.9200828157349896
Recall: 0.7688581314878893
F1 Score: 0.8377002827521206
Confusion Matrix:
 [[6917  193]
 [ 668 2222]]

Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.97      0.94      7110
           1       0.92      0.77      0.84      2890

    accuracy                           0.91     10000
   macro avg       0.92      0.87      0.89     10000
weighted avg       0.91      0.91      0.91     10000



Insights

- Random Forest baseline beats Logistic Regression in accuracy (expected, RF is more flexible).
- Recall for minority class (churn) is still lower — the problem of imbalance.

In [26]:
rf_bal = RandomForestClassifier(random_state=42, class_weight='balanced')


In [27]:
rf_bal.fit(X_train, y_train)


RandomForestClassifier(class_weight='balanced', random_state=42)

In [28]:
y_pred_rf_bal = rf_bal.predict(X_test)

In [29]:
# Evaluation 
print("Accuracy:", accuracy_score(y_test, y_pred_rf_bal))
print("Precision:", precision_score(y_test, y_pred_rf_bal))
print("Recall:", recall_score(y_test, y_pred_rf_bal))
print("F1 Score:", f1_score(y_test, y_pred_rf_bal))

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_rf_bal))
print("\nClassification Report:\n", classification_report(y_test, y_pred_rf_bal))


Accuracy: 0.9123
Precision: 0.9255813953488372
Recall: 0.757439446366782
F1 Score: 0.8331113225499525
Confusion Matrix:
 [[6934  176]
 [ 701 2189]]

Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.98      0.94      7110
           1       0.93      0.76      0.83      2890

    accuracy                           0.91     10000
   macro avg       0.92      0.87      0.89     10000
weighted avg       0.91      0.91      0.91     10000



#### Insights:
- The model predicts non-churn customers very well, but misses of churned customers.
- Applying class weights maintained precision but did not improve recall significantly.

In [30]:
param_grid = {
    'n_estimators': [100, 200],        # Number of trees
    'max_depth': [None, 10, 20],       # Depth of each tree
    'min_samples_split': [2, 5],       # Min samples to split node
    'min_samples_leaf': [1, 2],         # Min samples at leaf
    'max_features': ['sqrt'],# Features considered at split
    }

In [31]:

# Initialize base RF
rf = RandomForestClassifier(random_state=42)

# GridSearchCV
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=3,               # 3-fold cross-validation
    scoring='f1',       # Optimize for F1 (balance precision & recall)
    n_jobs=-1,
    verbose=2
)

# Fit
grid_search.fit(X_train, y_train)

# Best parameters
print("Best Parameters:", grid_search.best_params_)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
Best Parameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}


In [32]:
rf_tuned = grid_search.best_estimator_
# Predict
y_pred_rf_tuned = rf_tuned.predict(X_test)

In [33]:
print("Accuracy:", accuracy_score(y_test, y_pred_rf_tuned))
print("Precision:", precision_score(y_test, y_pred_rf_tuned))
print("Recall:", recall_score(y_test, y_pred_rf_tuned))
print("F1 Score:", f1_score(y_test, y_pred_rf_tuned))

print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_rf_tuned))
print("\nClassification Report:\n", classification_report(y_test, y_pred_rf_tuned))

Accuracy: 0.9176
Precision: 0.9251028806584363
Recall: 0.7778546712802769
F1 Score: 0.8451127819548873

Confusion Matrix:
 [[6928  182]
 [ 642 2248]]

Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.97      0.94      7110
           1       0.93      0.78      0.85      2890

    accuracy                           0.92     10000
   macro avg       0.92      0.88      0.89     10000
weighted avg       0.92      0.92      0.92     10000



#### insight
The tuned Random Forest model shows strong and balanced performance with 91.76% accuracy and an F1-score of 0.85. The model achieves high precision (0.92), indicating that most customers predicted as churn are correct. The recall of 0.78 shows the model successfully identifies a large proportion of churned customers, which is important for customer retention strategies. Overall, the tuned model improves churn detection while maintaining reliable overall classification performance.

## gradient boosting classifier

In [34]:
# Step 1: Initialize Gradient Boosting (baseline)
gbc_baseline = GradientBoostingClassifier(random_state=42)  # default params

# Step 2: Train the model
gbc_baseline.fit(X_train, y_train)

# Step 3: Make predictions
y_pred_gbc_baseline = gbc_baseline.predict(X_test)

# Step 4: Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred_gbc_baseline))
print("Precision:", precision_score(y_test, y_pred_gbc_baseline))
print("Recall:", recall_score(y_test, y_pred_gbc_baseline))
print("F1 Score:", f1_score(y_test, y_pred_gbc_baseline))

print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_gbc_baseline))
print("\nClassification Report:\n", classification_report(y_test, y_pred_gbc_baseline))

Accuracy: 0.9193
Precision: 0.9374212515749685
Recall: 0.772318339100346
F1 Score: 0.8468981217985202

Confusion Matrix:
 [[6961  149]
 [ 658 2232]]

Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.98      0.95      7110
           1       0.94      0.77      0.85      2890

    accuracy                           0.92     10000
   macro avg       0.93      0.88      0.90     10000
weighted avg       0.92      0.92      0.92     10000



In [35]:
gb = GradientBoostingClassifier(random_state=42)
param_grid_gb = {
    'n_estimators': [100, 200],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 5],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

In [36]:
grid_gb = GridSearchCV(
    estimator=gb,
    param_grid=param_grid_gb,
    cv=3,
    scoring='f1',
    n_jobs=-1,
    verbose=2
)

In [37]:
grid_gb.fit(X_train, y_train)
print("Best Parameters:", grid_gb.best_params_)

Fitting 3 folds for each of 32 candidates, totalling 96 fits
Best Parameters: {'learning_rate': 0.05, 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}


In [38]:
best_gb = grid_gb.best_estimator_
y_pred_gb_tuned = best_gb.predict(X_test)

In [39]:
print("Accuracy:", accuracy_score(y_test, y_pred_gb_tuned))
print("Precision:", precision_score(y_test, y_pred_gb_tuned))
print("Recall:", recall_score(y_test, y_pred_gb_tuned))
print("F1 Score:", f1_score(y_test, y_pred_gb_tuned))

print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_gb_tuned))
print("\nClassification Report:\n", classification_report(y_test, y_pred_gb_tuned))

Accuracy: 0.923
Precision: 0.9326530612244898
Recall: 0.7906574394463668
F1 Score: 0.8558052434456929

Confusion Matrix:
 [[6945  165]
 [ 605 2285]]

Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.98      0.95      7110
           1       0.93      0.79      0.86      2890

    accuracy                           0.92     10000
   macro avg       0.93      0.88      0.90     10000
weighted avg       0.92      0.92      0.92     10000



#### Insight – 
The tuned Gradient Boosting model outperforms the tuned Random Forest, achieving higher accuracy (92.32%) and better recall (0.79 vs 0.78), leading to an improved F1-score (0.86). This indicates stronger churn detection while maintaining high overall prediction performance.

# ada boost

In [40]:
ada=AdaBoostClassifier(random_state=42)
ada.fit(X_train,y_train)
a= ada.predict(X_test)


In [41]:
# Evaluation 
print("Accuracy:", accuracy_score(y_test, a))
print("Precision:", precision_score(y_test, a))
print("Recall:", recall_score(y_test, a))
print("F1 Score:", f1_score(y_test, a))

print("Confusion Matrix:\n", confusion_matrix(y_test, a))
print("\nClassification Report:\n", classification_report(y_test, a))


Accuracy: 0.8214
Precision: 0.7690058479532164
Recall: 0.5460207612456748
F1 Score: 0.6386078510724403
Confusion Matrix:
 [[6636  474]
 [1312 1578]]

Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.93      0.88      7110
           1       0.77      0.55      0.64      2890

    accuracy                           0.82     10000
   macro avg       0.80      0.74      0.76     10000
weighted avg       0.82      0.82      0.81     10000



 ### final insight

Among all trained models, Tuned Gradient Boosting Classifier delivered the best overall performance, achieving the highest balance of accuracy, recall, and F1-score.

The model is particularly effective at identifying churn customers, reducing false negatives and improving early churn detection.

Random Forest performed strongly as well but was slightly less effective in capturing difficult churn cases compared to Gradient Boosting.

Simpler models like Logistic Regression as good baselines but were limited in handling class imbalance and complex patterns.